In [6]:
from sklearn.model_selection import train_test_split
import pandas as pd
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping

### Get Data ###

In [ ]:
df = pd.read_csv('skin_cancer.csv')
X = df['image'].values
y = df['target'].values

### Split Data ###

In [ ]:
X_train, X_val_test, y_train, y_val_test = train_test_split(X, y, test_size=0.3)
X_val, X_test, y_val, y_test = train_test_split(X_val_test, y_val_test, test_size=0.5)

### CNN + MLP ###

In [ ]:

# CNN Model
def CNN_model(input_shape=(128, 128, 3)):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False  # Optional: fine-tune later
    
    x = MaxPooling2D()(base_model.output)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    
    return Model(inputs=base_model.input, outputs=x, name='cnn_model')


# MLP Model
def MLP_model(input_dim):
    inputs = layers.Input(shape=(input_dim,))
    x = layers.Dense(64, activation='relu')(inputs)
    x = layers.Dropout(0.3)(x)
    x = layers.Dense(32, activation='relu')(x)
    
    return models.Model(inputs=inputs, outputs=x, name='metadata_model')


# Combined Model
def build_combined_model(image_shape, metadata_dim):
    cnn_model = CNN_model(image_shape)
    mlp_model = MLP_model(metadata_dim)

    # Fusion
    combined = layers.concatenate([cnn_model.output, mlp_model.output])
    x = layers.Dense(64, activation='relu')(combined)
    x = layers.Dropout(0.3)(x)
    output = layers.Dense(1, activation='sigmoid')  # Binary classification

    model = models.Model(inputs=[cnn_model.input, mlp_model.input], outputs=output)
    return model


In [ ]:
model = build_combined_model(image_shape=(128, 128, 3), metadata_dim=55)

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit([X_train_image, X_train_meta], y, epochs=1, batch_size=256, shuffle=True, validation_data=(X_val, X_val), callback=[early_stopping_callback])

### Auto Encoder ###

# Define the convolutional autoencoder architecture
input_shape = X_train.shape[1:]

# Input layer
input_layer = Input(shape=input_shape)

# Encoding layers
x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
x = MaxPooling2D((2, 2), padding='same')(x)
x = Conv2D(16, (3, 3), activation='relu', padding='same')(x)
encoded = MaxPooling2D((2, 2), padding='same')(x)

# Decoding layers
x = Conv2D(16, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(input_shape[2], (3, 3), activation='sigmoid', padding='same')(x)

# Autoencoder model
autoencoder = Model(input_layer, decoded)

# Encoder model
encoder = Model(input_layer, encoded)

# Compile the autoencoder
autoencoder.compile(optimizer='adam', loss='mse')

# Early stopping
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the autoencoder
autoencoder.fit(X_train, X_train, epochs=1, batch_size=256, shuffle=True, validation_data=(X_val, X_val), callback=[early_stopping_callback])

# Encode the validation data
encoded_val = encoder.predict(X_val)

# Detect anomalies using reconstruction loss
reconstructed_val = autoencoder.predict(X_val)
reconstruction_error = np.mean((X_val - reconstructed_val) ** 2, axis=(1, 2, 3))

# Set a threshold for anomaly detection
threshold = reconstruction_error.mean() + 2 * reconstruction_error.std()

# Identify anomalies
anomalies = reconstruction_error > threshold
print("Number of anomalies detected:", anomalies.sum())